## Imports, data loading, model loading, etc.

In [1]:
import json
import os
import argparse

import numpy as np
import random
import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoImageProcessor, AutoModelForImageClassification, AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from pathlib import Path
from torch.utils.data import DataLoader, Subset
import sys
sys.path.append('../../lib/exlib/src')
from exlib.modules.sop import SOPConfig, SOPTextCls, get_chained_attr, compress_masks_text, AttributionOutputSOP

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED = 42
if SEED != -1:
    # Torch RNG
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    # Python RNG
    np.random.seed(SEED)
    random.seed(SEED)

In [3]:
# model paths
backbone_model_name = '../../pt_models/multirc_vanilla/best'
backbone_processor_name = 'bert-base-uncased'
# sop_config_path = 'configs/imagenet_m.json'

# data paths
# TRAIN_DATA_DIR = '../data/imagenet_m/train'
# VAL_DATA_DIR = '../data/imagenet_m/val'

# training args
batch_size = 1
lr = 0.000005
num_epochs = 20
warmup_steps = 2000
mask_batch_size = 4

# experiment args
exp_dir = '../../exps/multirc/best'

In [4]:
backbone_model = AutoModelForSequenceClassification.from_pretrained(backbone_model_name)
processor = AutoTokenizer.from_pretrained(backbone_processor_name)
backbone_config = AutoConfig.from_pretrained(backbone_model_name)

config = SOPConfig(json_file=os.path.join(exp_dir, 'config.json'),
                   projected_input_scale=2)

In [5]:
from torch.utils.data import DataLoader
from datasets import load_dataset

SENT_SEPS = [processor.convert_tokens_to_ids(processor.tokenize(token)[0]) for token in [';',',','.','?','!',';']]
SEP = processor.convert_tokens_to_ids(processor.tokenize('[SEP]')[0])
print('SEP', SEP, 'SENT_SEPS', SENT_SEPS)

def sent_seg(input_ids):
    segs = []
    count = 1
    for i, input_id in enumerate(input_ids):
        if count in [0, -1]:
            if input_id == SEP:
                count = -1
            segs.append(count)
            continue
        else:
            if input_id in SENT_SEPS:
                segs.append(count)
                count += 1
            elif input_id == SEP:
                if count > 0:
                    count = 0
                    segs.append(count)
                else:
                    segs.append(count)
                    count = -1
            else: # normal character
                segs.append(count)
    return segs

def convert_idx_masks_to_bool_text(masks):
    """
    input: masks (1, seq_len)
    output: masks_bool (num_masks, seq_len)
    """
    unique_idxs = torch.sort(torch.unique(masks)).values
    unique_idxs = unique_idxs[unique_idxs != -1]
    unique_idxs = unique_idxs[unique_idxs != 0]
    idxs = unique_idxs.view(-1, 1)
    broadcasted_masks = masks.expand(unique_idxs.shape[0], 
                                     masks.shape[1])
    masks_bool = (broadcasted_masks == idxs)
    return masks_bool

def get_mask_transform_text(num_masks_max=200, processor=None):
    def mask_transform(mask):
        seg_mask_cut_off = num_masks_max
        # print('mask 1', mask)
        # if mask.max(dim=-1) > seg_mask_cut_off:
        # import pdb; pdb.set_trace()
        if mask.max(dim=-1).values.item() > seg_mask_cut_off:
            mask_new = (mask / (mask.max(dim=-1).values / seg_mask_cut_off)).int().float() + 1
            # bsz, seq_len = mask_new.shape
            # print('mask 2', mask_new)
            # import pdb; pdb.set_trace()
            mask_new[mask == 0] = 0
            mask_new[mask == -1] = -1
            mask = mask_new
        
        if mask.dtype != torch.bool:
            if len(mask.shape) == 1:
                mask = mask.unsqueeze(0)
            # print('mask', mask.shape)
            mask_bool = convert_idx_masks_to_bool_text(mask)
        # print(mask.shape)
        bsz, seq_len = mask.shape
        mask_bool = mask_bool.float()
        
        

        if bsz < seg_mask_cut_off:
            repeat_count = seg_mask_cut_off // bsz + 1
            mask_bool = torch.cat([mask_bool] * repeat_count, dim=0)

        # add additional mask afterwards
        mask_bool_sum = torch.sum(mask_bool[:seg_mask_cut_off - 1], dim=0, keepdim=True).bool()
        if False in mask_bool_sum:
            mask_bool = mask_bool[:seg_mask_cut_off - 1]
            # import pdb; pdb.set_trace()
            compensation_mask = (1 - mask_bool_sum.int()).bool()
            compensation_mask[mask == 0] = False
            compensation_mask[mask == -1] = False
            mask_bool = torch.cat([mask_bool, compensation_mask])
        else:
            mask_bool = mask_bool[:seg_mask_cut_off]
        return mask_bool
    return mask_transform

mask_transform = get_mask_transform_text(config.num_masks_max)

def transform(batch):
    # Preprocess the image using the ViTImageProcessor
    if processor is not None:
        inputs = processor(batch['passage'], 
                           batch['query_and_answer'], 
                           padding='max_length', 
                           truncation=True, 
                           max_length=512)
        segs = [sent_seg(input_id) for input_id in inputs['input_ids']]
        inputs = {k: torch.tensor(v) for k, v in inputs.items()}
        
        segs_bool = []
        for seg in segs:
            seg_bool = mask_transform(torch.tensor(seg))
            segs_bool.append(seg_bool)
        inputs['segs'] = torch.stack(segs_bool)
        # print("inputs['segs']", inputs['segs'].shape)
        # for k, v in inputs.items():
        #     print(k, v.shape)
        # import pdb; pdb.set_trace()
        return inputs
    else:
        return batch


# train_size, val_size = -1, -1
train_size, val_size = 100, 100

train_dataset = load_dataset('eraser_multi_rc', split='train')
train_dataset = train_dataset.map(transform, batched=True,
                            remove_columns=['passage', 
                                            'query_and_answer',
                                            'evidences'])

val_dataset = load_dataset('eraser_multi_rc', split='validation')
val_dataset = val_dataset.map(transform, batched=True,
                            remove_columns=['passage', 
                                            'query_and_answer',
                                            'evidences'])

if train_size != -1:
    train_dataset = Subset(train_dataset, list(range(train_size)))
if val_size != -1:
    val_dataset = Subset(val_dataset, list(range(val_size)))

# Create a DataLoader to batch and shuffle the data
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

SEP 102 SENT_SEPS [1025, 1010, 1012, 1029, 999, 1025]


Generating train split:   0%|          | 0/24029 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3214 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4848 [00:00<?, ? examples/s]

Map:   0%|          | 0/24029 [00:00<?, ? examples/s]

Map:   0%|          | 0/3214 [00:00<?, ? examples/s]

In [6]:
from collections import namedtuple

WrappedBackboneOutput = namedtuple("WrappedBackboneOutput", 
                                  ["logits",
                                   "pooler_output"])


class WrappedBackboneModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, inputs=None, **kwargs):
        outputs = self.model(inputs, output_hidden_states=True, **kwargs)
        return WrappedBackboneOutput(outputs.logits, outputs.hidden_states[-1][:,0])

In [7]:
wrapped_backbone_model = WrappedBackboneModel(backbone_model)
wrapped_backbone_model = wrapped_backbone_model.to(device)
class_weights = get_chained_attr(wrapped_backbone_model, config.finetune_layers[0]).weight #.clone().to(device)
projection_layer = wrapped_backbone_model.model.bert.embeddings.word_embeddings

In [8]:
model = SOPTextCls(config, wrapped_backbone_model, class_weights=class_weights, projection_layer=projection_layer)
state_dict = torch.load(os.path.join(exp_dir, 'checkpoint.pth'))
model.load_state_dict(state_dict['model'])
model = model.to(device)
model.eval();

deep copy class weights


In [9]:
batch = next(iter(val_dataloader))
print(batch.keys())

dict_keys(['label', 'input_ids', 'token_type_ids', 'attention_mask', 'segs'])


In [10]:
if not isinstance(batch['input_ids'], torch.Tensor):
    inputs = torch.stack(batch['input_ids']).transpose(0, 1).to(device)
    if 'token_type_ids' in batch:
        token_type_ids = torch.stack(batch['token_type_ids']).transpose(0, 1).to(device)
    else:
        token_type_ids = None
    attention_mask = torch.stack(batch['attention_mask']).transpose(0, 1).to(device)

    concatenated_rows = [torch.stack(sublist) for sublist in batch['segs']]
    segs = torch.stack(concatenated_rows).permute(2, 0, 1).to(device).float()
    # print('segs', segs.shape)
else:
    inputs = batch['input_ids'].to(device)
    if 'token_type_ids' in batch:
        token_type_ids = batch['token_type_ids'].to(device)
    else:
        token_type_ids = None
    attention_mask = batch['attention_mask'].to(device)
    segs = batch['segs'].to(device).float()
kwargs = {
    'token_type_ids': token_type_ids,
    'attention_mask': attention_mask,
}
labels = batch['label'].to(device)

## Start here to get explanations

In [11]:
expln = model(inputs, segs=segs, kwargs=kwargs, return_tuple=True)

In [12]:
expln._fields

('logits',
 'logits_all',
 'pooler_outputs_all',
 'masks',
 'mask_weights',
 'attributions',
 'attributions_max',
 'attributions_all',
 'flat_masks',
 'grouped_attributions')

In [16]:
expln.logits.shape, expln.pooler_outputs_all.shape, expln.masks.shape, expln.mask_weights.shape, expln.grouped_attributions.shape

(torch.Size([1, 2]),
 torch.Size([1, 16, 768]),
 torch.Size([1, 16, 512]),
 torch.Size([1, 16, 2]),
 torch.Size([1, 16, 2]))

In [14]:
expln.grouped_attributions

tensor([[[ 0.2403, -0.1948],
         [ 0.2322, -0.1894],
         [ 0.2367, -0.1920],
         [ 0.2355, -0.1913],
         [ 0.2313, -0.1911],
         [ 0.2408, -0.1992],
         [ 0.2381, -0.1992],
         [ 0.2383, -0.2004],
         [ 0.2388, -0.1999],
         [ 0.2237, -0.1980],
         [ 0.2328, -0.1883],
         [ 0.2340, -0.1875],
         [ 0.2370, -0.1987],
         [ 0.2374, -0.1941],
         [ 0.2387, -0.1948],
         [ 0.2367, -0.1969]]], device='cuda:0', grad_fn=<MulBackward0>)

In [17]:
expln.mask_weights

tensor([[[0.0630, 0.0611],
         [0.0628, 0.0640],
         [0.0629, 0.0625],
         [0.0625, 0.0632],
         [0.0629, 0.0647],
         [0.0629, 0.0615],
         [0.0623, 0.0612],
         [0.0625, 0.0624],
         [0.0626, 0.0617],
         [0.0586, 0.0644],
         [0.0627, 0.0635],
         [0.0630, 0.0627],
         [0.0626, 0.0612],
         [0.0629, 0.0626],
         [0.0632, 0.0623],
         [0.0627, 0.0610]]], device='cuda:0', grad_fn=<TransposeBackward0>)

In [20]:
expln.grouped_attributions.sum(dim=-2) - expln.logits

tensor([[0.0000e+00, 2.3842e-07]], device='cuda:0', grad_fn=<SubBackward0>)

In [25]:
expln.masks.shape

torch.Size([1, 16, 512])

In [26]:
expln.masks[0][7]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.4352, 0.4352, 0.4352, 0.4352, 0.4352, 0.4352, 0.4352, 0.4352,
        0.4352, 0.4352, 0.4352, 0.4352, 0.4352, 0.4352, 0.4352, 0.4352, 0.4352,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0115, 0.0115, 

In [27]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))


In [28]:
printmd("<span style='background-color:rgb(255,0,0)'>hi</span> hi")

<span style='background-color:rgb(255,0,0)'>hi</span> hi

In [29]:
print(inputs.shape)
tokens = processor.convert_ids_to_tokens(inputs[0])
len(tokens)

torch.Size([1, 512])


512

In [30]:
print_str = ""
for i in range(len(tokens)):
    expln_val = expln.masks[0][0][i].item()
    if expln_val == 0:
        print_str += f" {tokens[i]}"
    else:
        print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"

In [31]:
printmd(print_str) # cyan is 1, red is low. white is not used at all.

 [CLS] allan sat down at his desk and pulled the chair in close . opening a side drawer , he took out a piece of paper and his ink ##pot . after filling his pen , allan looked at his paper in the orange glow from the lantern set back in the desk ' s right - hand corner . his pen cast a forbid ##ding line of shadow slant ##ing across the page , echoing the ink ##y darkness crouch ##ing in the edges of the lantern ' s struggling glow . the only other illumination came from a lu ##rid moonlight filtered through thin branches and clouds , casting its bone - pale glow onto the pine floor ##boards . allan unfolded another page ,<span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> this</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> one</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> crowded</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> with</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> ranks</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> of</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> letters</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> in</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> tight</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> formation</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> from</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> left</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> to</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> right</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> .</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> the</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> lines</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> of</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> letters</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> stepped</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> into</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> their</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> divisions</span><span style='background-color:rgb(204.82764802873135,50.172351971268654,50.172351971268654)'> ,</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> in</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> the</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> shape</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> of</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> a</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> story</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> '</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> s</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> outline</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> :</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> the</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> loose</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> ,</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> dry</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> skeleton</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> of</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> a</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> tale</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> lay</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> exposed</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> beneath</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> their</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> feet</span><span style='background-color:rgb(237.04670816659927,17.953291833400726,17.953291833400726)'> ,</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> awaiting</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> tend</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> ##ons</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> ,</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> muscle</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> and</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> blushing</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> skin</span><span style='background-color:rgb(93.46536308526993,161.53463691473007,161.53463691473007)'> .</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> allan</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> reviewed</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> the</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> troops</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> ,</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> all</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> prepared</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> to</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> di</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> ##se</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> ##mba</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> ##rk</span><span style='background-color:rgb(138.0333311855793,116.9666688144207,116.9666688144207)'> ,</span><span style='background-color:rgb(0.0,255.0,255.0)'> their</span><span style='background-color:rgb(0.0,255.0,255.0)'> task</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> form</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> tale</span><span style='background-color:rgb(0.0,255.0,255.0)'> of</span><span style='background-color:rgb(0.0,255.0,255.0)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> young</span><span style='background-color:rgb(0.0,255.0,255.0)'> man</span><span style='background-color:rgb(0.0,255.0,255.0)'> returning</span><span style='background-color:rgb(0.0,255.0,255.0)'> home</span><span style='background-color:rgb(0.0,255.0,255.0)'> from</span><span style='background-color:rgb(0.0,255.0,255.0)'> life</span><span style='background-color:rgb(0.0,255.0,255.0)'> abroad</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> find</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(0.0,255.0,255.0)'> childhood</span><span style='background-color:rgb(0.0,255.0,255.0)'> friend</span><span style='background-color:rgb(0.0,255.0,255.0)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> bride</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> -</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> ,</span><span style='background-color:rgb(0.0,255.0,255.0)'> thus</span><span style='background-color:rgb(0.0,255.0,255.0)'> upset</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##ting</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> apple</span><span style='background-color:rgb(0.0,255.0,255.0)'> cart</span><span style='background-color:rgb(0.0,255.0,255.0)'> of</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(0.0,255.0,255.0)'> life</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> plan</span><span style='background-color:rgb(0.0,255.0,255.0)'> ,</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> clarified</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> –</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> of</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> course</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> –</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> by</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> his</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> very</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> time</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> away</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> from</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> her</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> he</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> loved</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> best</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> .</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> although</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> the</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> concept</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> was</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> a</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> simple</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> one</span><span style='background-color:rgb(239.68237878754735,15.31762121245265,15.31762121245265)'> ,</span><span style='background-color:rgb(132.49122977256775,122.50877022743225,122.50877022743225)'> allan</span><span style='background-color:rgb(132.49122977256775,122.50877022743225,122.50877022743225)'> thought</span><span style='background-color:rgb(132.49122977256775,122.50877022743225,122.50877022743225)'> it</span><span style='background-color:rgb(132.49122977256775,122.50877022743225,122.50877022743225)'> had</span><span style='background-color:rgb(132.49122977256775,122.50877022743225,122.50877022743225)'> potential</span><span style='background-color:rgb(132.49122977256775,122.50877022743225,122.50877022743225)'> .</span><span style='background-color:rgb(132.49122977256775,122.50877022743225,122.50877022743225)'> besides</span><span style='background-color:rgb(132.49122977256775,122.50877022743225,122.50877022743225)'> ,</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> the</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> public</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> liked</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> a</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> good</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> ,</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> simple</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> romance</span><span style='background-color:rgb(113.09314906597137,141.90685093402863,141.90685093402863)'> .</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> perhaps</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> this</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> will</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> be</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> more</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> sale</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> ##able</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> ,</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> he</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> thought</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> and</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> began</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> to</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> write</span><span style='background-color:rgb(103.99482607841492,151.00517392158508,151.00517392158508)'> .</span> they gazed at each other , lost in the rap ##ture of love based so deeply within their hearts that they had never seen it before .<span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> "</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> what</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> about</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> roger</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> ?</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> "</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> she</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> asked</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> ,</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> knowing</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> that</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> the</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> answer</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> no</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> longer</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> mattered</span><span style='background-color:rgb(108.06051671504974,146.93948328495026,146.93948328495026)'> .</span> [SEP]<span style='background-color:rgb(0.0,255.0,255.0)'> name</span><span style='background-color:rgb(0.0,255.0,255.0)'> few</span><span style='background-color:rgb(0.0,255.0,255.0)'> objects</span><span style='background-color:rgb(0.0,255.0,255.0)'> said</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> in</span><span style='background-color:rgb(0.0,255.0,255.0)'> or</span><span style='background-color:rgb(0.0,255.0,255.0)'> on</span><span style='background-color:rgb(0.0,255.0,255.0)'> allan</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> desk</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##r</span><span style='background-color:rgb(0.0,255.0,255.0)'> [SEP]</span> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [32]:
expln.mask_weights.shape

torch.Size([1, 16, 2])

In [22]:
for j in range(expln.masks.shape[1]):
    print(expln.mask_weights[0][j])
    print_str = ""
    for i in range(len(tokens)):
        expln_val = expln.masks[0][j][i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    printmd(print_str)

tensor([0.0630, 0.0611], device='cuda:0', grad_fn=<SelectBackward0>)


 [CLS] allan sat down at his desk and pulled the chair in close . opening a side drawer , he took out a piece of paper and his ink ##pot . after filling his pen , allan looked at his paper in the orange glow from the lantern set back in the desk ' s right - hand corner . his pen cast a forbid ##ding line of shadow slant ##ing across the page , echoing the ink ##y darkness crouch ##ing in the edges of the lantern ' s struggling glow . the only other illumination came from a lu ##rid moonlight filtered through thin branches and clouds , casting its bone - pale glow onto the pine floor ##boards . allan unfolded another page ,<span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> this</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> one</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> crowded</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> with</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> ranks</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> of</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> letters</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> in</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> tight</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> formation</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> from</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> left</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> to</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> right</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> .</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> the</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> lines</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> of</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> letters</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> stepped</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> into</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> their</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> divisions</span><span style='background-color:rgb(204.82740864157677,50.17259135842323,50.17259135842323)'> ,</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> in</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> the</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> shape</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> of</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> a</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> story</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> '</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> s</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> outline</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> :</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> the</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> loose</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> ,</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> dry</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> skeleton</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> of</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> a</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> tale</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> lay</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> exposed</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> beneath</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> their</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> feet</span><span style='background-color:rgb(237.04619519412518,17.953804805874825,17.953804805874825)'> ,</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> awaiting</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> tend</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> ##ons</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> ,</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> muscle</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> and</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> blushing</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> skin</span><span style='background-color:rgb(93.46537828445435,161.53462171554565,161.53462171554565)'> .</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> allan</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> reviewed</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> the</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> troops</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> ,</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> all</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> prepared</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> to</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> di</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> ##se</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> ##mba</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> ##rk</span><span style='background-color:rgb(138.03336158394814,116.96663841605186,116.96663841605186)'> ,</span><span style='background-color:rgb(0.0,255.0,255.0)'> their</span><span style='background-color:rgb(0.0,255.0,255.0)'> task</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> form</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> tale</span><span style='background-color:rgb(0.0,255.0,255.0)'> of</span><span style='background-color:rgb(0.0,255.0,255.0)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> young</span><span style='background-color:rgb(0.0,255.0,255.0)'> man</span><span style='background-color:rgb(0.0,255.0,255.0)'> returning</span><span style='background-color:rgb(0.0,255.0,255.0)'> home</span><span style='background-color:rgb(0.0,255.0,255.0)'> from</span><span style='background-color:rgb(0.0,255.0,255.0)'> life</span><span style='background-color:rgb(0.0,255.0,255.0)'> abroad</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> find</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(0.0,255.0,255.0)'> childhood</span><span style='background-color:rgb(0.0,255.0,255.0)'> friend</span><span style='background-color:rgb(0.0,255.0,255.0)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> bride</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> -</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> ,</span><span style='background-color:rgb(0.0,255.0,255.0)'> thus</span><span style='background-color:rgb(0.0,255.0,255.0)'> upset</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##ting</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> apple</span><span style='background-color:rgb(0.0,255.0,255.0)'> cart</span><span style='background-color:rgb(0.0,255.0,255.0)'> of</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(0.0,255.0,255.0)'> life</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> plan</span><span style='background-color:rgb(0.0,255.0,255.0)'> ,</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> clarified</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> –</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> of</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> course</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> –</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> by</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> his</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> very</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> time</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> away</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> from</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> her</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> he</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> loved</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> best</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> .</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> although</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> the</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> concept</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> was</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> a</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> simple</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> one</span><span style='background-color:rgb(239.68299055472016,15.317009445279837,15.317009445279837)'> ,</span><span style='background-color:rgb(132.4915337562561,122.5084662437439,122.5084662437439)'> allan</span><span style='background-color:rgb(132.4915337562561,122.5084662437439,122.5084662437439)'> thought</span><span style='background-color:rgb(132.4915337562561,122.5084662437439,122.5084662437439)'> it</span><span style='background-color:rgb(132.4915337562561,122.5084662437439,122.5084662437439)'> had</span><span style='background-color:rgb(132.4915337562561,122.5084662437439,122.5084662437439)'> potential</span><span style='background-color:rgb(132.4915337562561,122.5084662437439,122.5084662437439)'> .</span><span style='background-color:rgb(132.4915337562561,122.5084662437439,122.5084662437439)'> besides</span><span style='background-color:rgb(132.4915337562561,122.5084662437439,122.5084662437439)'> ,</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> the</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> public</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> liked</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> a</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> good</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> ,</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> simple</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> romance</span><span style='background-color:rgb(113.09289067983627,141.90710932016373,141.90710932016373)'> .</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> perhaps</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> this</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> will</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> be</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> more</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> sale</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> ##able</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> ,</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> he</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> thought</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> and</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> began</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> to</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> write</span><span style='background-color:rgb(103.99485647678375,151.00514352321625,151.00514352321625)'> .</span> they gazed at each other , lost in the rap ##ture of love based so deeply within their hearts that they had never seen it before .<span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> "</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> what</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> about</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> roger</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> ?</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> "</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> she</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> asked</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> ,</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> knowing</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> that</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> the</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> answer</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> no</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> longer</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> mattered</span><span style='background-color:rgb(108.06025832891464,146.93974167108536,146.93974167108536)'> .</span> [SEP]<span style='background-color:rgb(0.0,255.0,255.0)'> name</span><span style='background-color:rgb(0.0,255.0,255.0)'> few</span><span style='background-color:rgb(0.0,255.0,255.0)'> objects</span><span style='background-color:rgb(0.0,255.0,255.0)'> said</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> in</span><span style='background-color:rgb(0.0,255.0,255.0)'> or</span><span style='background-color:rgb(0.0,255.0,255.0)'> on</span><span style='background-color:rgb(0.0,255.0,255.0)'> allan</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> desk</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##r</span><span style='background-color:rgb(0.0,255.0,255.0)'> [SEP]</span> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

tensor([0.0628, 0.0640], device='cuda:0', grad_fn=<SelectBackward0>)


 [CLS] allan sat down at his desk and pulled the chair in close . opening a side drawer ,<span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> he</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> took</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> out</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> a</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> piece</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> of</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> paper</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> and</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> his</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> ink</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> ##pot</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> .</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> after</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> filling</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> his</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> pen</span><span style='background-color:rgb(236.33256118744612,18.667438812553883,18.667438812553883)'> ,</span> allan looked at his paper in the orange glow from the lantern set back in the desk ' s right - hand corner . his pen cast a forbid ##ding line of shadow slant ##ing across the page , echoing the ink ##y darkness crouch ##ing in the edges of the lantern ' s struggling glow . the only other illumination came from a lu ##rid moonlight filtered through thin branches and clouds , casting its bone - pale glow onto the pine floor ##boards . allan unfolded another page ,<span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> this</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> one</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> crowded</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> with</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> ranks</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> of</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> letters</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> in</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> tight</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> formation</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> from</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> left</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> to</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> right</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> .</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> the</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> lines</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> of</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> letters</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> stepped</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> into</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> their</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> divisions</span><span style='background-color:rgb(238.20976875722408,16.790231242775917,16.790231242775917)'> ,</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> in</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> the</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> shape</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> of</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> a</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> story</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> '</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> s</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> outline</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> :</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> the</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> loose</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> ,</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> dry</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> skeleton</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> of</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> a</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> tale</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> lay</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> exposed</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> beneath</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> their</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> feet</span><span style='background-color:rgb(240.54123340174556,14.458766598254442,14.458766598254442)'> ,</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> awaiting</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> tend</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> ##ons</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> ,</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> muscle</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> and</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> blushing</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> skin</span><span style='background-color:rgb(186.10928624868393,68.89071375131607,68.89071375131607)'> .</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> allan</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> reviewed</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> the</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> troops</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> ,</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> all</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> prepared</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> to</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> di</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> ##se</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> ##mba</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> ##rk</span><span style='background-color:rgb(206.34527519345284,48.654724806547165,48.654724806547165)'> ,</span><span style='background-color:rgb(0.0,255.0,255.0)'> their</span><span style='background-color:rgb(0.0,255.0,255.0)'> task</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> form</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> tale</span><span style='background-color:rgb(0.0,255.0,255.0)'> of</span><span style='background-color:rgb(0.0,255.0,255.0)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> young</span><span style='background-color:rgb(0.0,255.0,255.0)'> man</span><span style='background-color:rgb(0.0,255.0,255.0)'> returning</span><span style='background-color:rgb(0.0,255.0,255.0)'> home</span><span style='background-color:rgb(0.0,255.0,255.0)'> from</span><span style='background-color:rgb(0.0,255.0,255.0)'> life</span><span style='background-color:rgb(0.0,255.0,255.0)'> abroad</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> find</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(0.0,255.0,255.0)'> childhood</span><span style='background-color:rgb(0.0,255.0,255.0)'> friend</span><span style='background-color:rgb(0.0,255.0,255.0)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> bride</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> -</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> ,</span><span style='background-color:rgb(0.0,255.0,255.0)'> thus</span><span style='background-color:rgb(0.0,255.0,255.0)'> upset</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##ting</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> apple</span><span style='background-color:rgb(0.0,255.0,255.0)'> cart</span><span style='background-color:rgb(0.0,255.0,255.0)'> of</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(0.0,255.0,255.0)'> life</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> plan</span><span style='background-color:rgb(0.0,255.0,255.0)'> ,</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> clarified</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> –</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> of</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> course</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> –</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> by</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> his</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> very</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> time</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> away</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> from</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> her</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> he</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> loved</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> best</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> .</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> although</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> the</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> concept</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> was</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> a</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> simple</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> one</span><span style='background-color:rgb(244.19595044106245,10.80404955893755,10.80404955893755)'> ,</span><span style='background-color:rgb(194.60729084908962,60.39270915091038,60.39270915091038)'> allan</span><span style='background-color:rgb(194.60729084908962,60.39270915091038,60.39270915091038)'> thought</span><span style='background-color:rgb(194.60729084908962,60.39270915091038,60.39270915091038)'> it</span><span style='background-color:rgb(194.60729084908962,60.39270915091038,60.39270915091038)'> had</span><span style='background-color:rgb(194.60729084908962,60.39270915091038,60.39270915091038)'> potential</span><span style='background-color:rgb(194.60729084908962,60.39270915091038,60.39270915091038)'> .</span><span style='background-color:rgb(194.60729084908962,60.39270915091038,60.39270915091038)'> besides</span><span style='background-color:rgb(194.60729084908962,60.39270915091038,60.39270915091038)'> ,</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> the</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> public</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> liked</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> a</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> good</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> ,</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> simple</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> romance</span><span style='background-color:rgb(190.05022197961807,64.94977802038193,64.94977802038193)'> .</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> perhaps</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> this</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> will</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> be</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> more</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> sale</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> ##able</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> ,</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> he</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> thought</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> and</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> began</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> to</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> write</span><span style='background-color:rgb(147.69595429301262,107.30404570698738,107.30404570698738)'> .</span> they gazed at each other , lost in the rap ##ture of love based so deeply within their hearts that they had never seen it before .<span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> "</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> what</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> about</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> roger</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> ?</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> "</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> she</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> asked</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> ,</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> knowing</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> that</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> the</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> answer</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> no</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> longer</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> mattered</span><span style='background-color:rgb(160.6208771467209,94.37912285327911,94.37912285327911)'> .</span> [SEP]<span style='background-color:rgb(0.0,255.0,255.0)'> name</span><span style='background-color:rgb(0.0,255.0,255.0)'> few</span><span style='background-color:rgb(0.0,255.0,255.0)'> objects</span><span style='background-color:rgb(0.0,255.0,255.0)'> said</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> in</span><span style='background-color:rgb(0.0,255.0,255.0)'> or</span><span style='background-color:rgb(0.0,255.0,255.0)'> on</span><span style='background-color:rgb(0.0,255.0,255.0)'> allan</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> desk</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##r</span><span style='background-color:rgb(0.0,255.0,255.0)'> [SEP]</span> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

tensor([0.0629, 0.0625], device='cuda:0', grad_fn=<SelectBackward0>)


...

 [CLS] allan sat down at his desk and pulled the chair in close . opening a side drawer ,<span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> he</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> took</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> out</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> a</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> piece</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> of</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> paper</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> and</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> his</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> ink</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> ##pot</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> .</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> after</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> filling</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> his</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> pen</span><span style='background-color:rgb(173.30354779958725,81.69645220041275,81.69645220041275)'> ,</span> allan looked at his paper in the orange glow from the lantern set back in the desk ' s right - hand corner . his pen cast a forbid ##ding line of shadow slant ##ing across the page , echoing the ink ##y darkness crouch ##ing in the edges of the lantern ' s struggling glow . the only other illumination came from a lu ##rid moonlight filtered through thin branches and clouds , casting its bone - pale glow onto the pine floor ##boards . allan unfolded another page ,<span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> this</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> one</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> crowded</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> with</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> ranks</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> of</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> letters</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> in</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> tight</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> formation</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> from</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> left</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> to</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> right</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> .</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> the</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> lines</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> of</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> letters</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> stepped</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> into</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> their</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> divisions</span><span style='background-color:rgb(241.74301771447062,13.256982285529375,13.256982285529375)'> ,</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> in</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> the</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> shape</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> of</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> a</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> story</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> '</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> s</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> outline</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> :</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> the</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> loose</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> ,</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> dry</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> skeleton</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> of</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> a</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> tale</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> lay</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> exposed</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> beneath</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> their</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> feet</span><span style='background-color:rgb(217.49201506376266,37.507984936237335,37.507984936237335)'> ,</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> awaiting</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> tend</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> ##ons</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> ,</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> muscle</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> and</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> blushing</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> skin</span><span style='background-color:rgb(150.17295017838478,104.82704982161522,104.82704982161522)'> .</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> allan</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> reviewed</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> the</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> troops</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> ,</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> all</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> prepared</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> to</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> di</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> ##se</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> ##mba</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> ##rk</span><span style='background-color:rgb(187.68627762794495,67.31372237205505,67.31372237205505)'> ,</span><span style='background-color:rgb(0.0,255.0,255.0)'> their</span><span style='background-color:rgb(0.0,255.0,255.0)'> task</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> form</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> tale</span><span style='background-color:rgb(0.0,255.0,255.0)'> of</span><span style='background-color:rgb(0.0,255.0,255.0)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> young</span><span style='background-color:rgb(0.0,255.0,255.0)'> man</span><span style='background-color:rgb(0.0,255.0,255.0)'> returning</span><span style='background-color:rgb(0.0,255.0,255.0)'> home</span><span style='background-color:rgb(0.0,255.0,255.0)'> from</span><span style='background-color:rgb(0.0,255.0,255.0)'> life</span><span style='background-color:rgb(0.0,255.0,255.0)'> abroad</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> find</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(0.0,255.0,255.0)'> childhood</span><span style='background-color:rgb(0.0,255.0,255.0)'> friend</span><span style='background-color:rgb(0.0,255.0,255.0)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> bride</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> -</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> ,</span><span style='background-color:rgb(0.0,255.0,255.0)'> thus</span><span style='background-color:rgb(0.0,255.0,255.0)'> upset</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##ting</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> apple</span><span style='background-color:rgb(0.0,255.0,255.0)'> cart</span><span style='background-color:rgb(0.0,255.0,255.0)'> of</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(0.0,255.0,255.0)'> life</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> plan</span><span style='background-color:rgb(0.0,255.0,255.0)'> ,</span> clarified – of course – by his very time away from her he loved best . although the concept was a simple one ,<span style='background-color:rgb(188.6854264140129,66.31457358598709,66.31457358598709)'> allan</span><span style='background-color:rgb(188.6854264140129,66.31457358598709,66.31457358598709)'> thought</span><span style='background-color:rgb(188.6854264140129,66.31457358598709,66.31457358598709)'> it</span><span style='background-color:rgb(188.6854264140129,66.31457358598709,66.31457358598709)'> had</span><span style='background-color:rgb(188.6854264140129,66.31457358598709,66.31457358598709)'> potential</span><span style='background-color:rgb(188.6854264140129,66.31457358598709,66.31457358598709)'> .</span><span style='background-color:rgb(188.6854264140129,66.31457358598709,66.31457358598709)'> besides</span><span style='background-color:rgb(188.6854264140129,66.31457358598709,66.31457358598709)'> ,</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> the</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> public</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> liked</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> a</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> good</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> ,</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> simple</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> romance</span><span style='background-color:rgb(181.46238639950752,73.53761360049248,73.53761360049248)'> .</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> perhaps</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> this</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> will</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> be</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> more</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> sale</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> ##able</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> ,</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> he</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> thought</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> and</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> began</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> to</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> write</span><span style='background-color:rgb(94.03803795576096,160.96196204423904,160.96196204423904)'> .</span> they gazed at each other , lost in the rap ##ture of love based so deeply within their hearts that they had never seen it before .<span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> "</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> what</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> about</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> roger</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> ?</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> "</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> she</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> asked</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> ,</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> knowing</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> that</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> the</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> answer</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> no</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> longer</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> mattered</span><span style='background-color:rgb(124.0026068687439,130.9973931312561,130.9973931312561)'> .</span> [SEP]<span style='background-color:rgb(0.0,255.0,255.0)'> name</span><span style='background-color:rgb(0.0,255.0,255.0)'> few</span><span style='background-color:rgb(0.0,255.0,255.0)'> objects</span><span style='background-color:rgb(0.0,255.0,255.0)'> said</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> in</span><span style='background-color:rgb(0.0,255.0,255.0)'> or</span><span style='background-color:rgb(0.0,255.0,255.0)'> on</span><span style='background-color:rgb(0.0,255.0,255.0)'> allan</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> desk</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##r</span><span style='background-color:rgb(0.0,255.0,255.0)'> [SEP]</span> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

tensor([0.0632, 0.0623], device='cuda:0', grad_fn=<SelectBackward0>)


 [CLS] allan sat down at his desk and pulled the chair in close . opening a side drawer ,<span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> he</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> took</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> out</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> a</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> piece</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> of</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> paper</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> and</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> his</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> ink</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> ##pot</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> .</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> after</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> filling</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> his</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> pen</span><span style='background-color:rgb(196.158789396286,58.84121060371399,58.84121060371399)'> ,</span> allan looked at his paper in the orange glow from the lantern set back in the desk ' s right - hand corner . his pen cast a forbid ##ding line of shadow slant ##ing across the page , echoing the ink ##y darkness crouch ##ing in the edges of the lantern ' s struggling glow . the only other illumination came from a lu ##rid moonlight filtered through thin branches and clouds , casting its bone - pale glow onto the pine floor ##boards . allan unfolded another page ,<span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> this</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> one</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> crowded</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> with</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> ranks</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> of</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> letters</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> in</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> tight</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> formation</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> from</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> left</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> to</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> right</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> .</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> the</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> lines</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> of</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> letters</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> stepped</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> into</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> their</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> divisions</span><span style='background-color:rgb(232.7106525376439,22.28934746235609,22.28934746235609)'> ,</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> in</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> the</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> shape</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> of</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> a</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> story</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> '</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> s</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> outline</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> :</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> the</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> loose</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> ,</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> dry</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> skeleton</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> of</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> a</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> tale</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> lay</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> exposed</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> beneath</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> their</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> feet</span><span style='background-color:rgb(210.91628551483154,44.08371448516846,44.08371448516846)'> ,</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> awaiting</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> tend</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> ##ons</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> ,</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> muscle</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> and</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> blushing</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> skin</span><span style='background-color:rgb(142.8872211277485,112.11277887225151,112.11277887225151)'> .</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> allan</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> reviewed</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> the</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> troops</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> ,</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> all</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> prepared</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> to</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> di</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> ##se</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> ##mba</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> ##rk</span><span style='background-color:rgb(171.14321172237396,83.85678827762604,83.85678827762604)'> ,</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> their</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> task</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> to</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> form</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> the</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> tale</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> of</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> a</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> young</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> man</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> returning</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> home</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> from</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> life</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> abroad</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> to</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> find</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> his</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> childhood</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> friend</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> a</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> bride</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> to</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> -</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> be</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> ,</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> thus</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> upset</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> ##ting</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> the</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> apple</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> cart</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> of</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> his</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> life</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> '</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> s</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> plan</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> ,</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> clarified</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> –</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> of</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> course</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> –</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> by</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> his</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> very</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> time</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> away</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> from</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> her</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> he</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> loved</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> best</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> .</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> although</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> the</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> concept</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> was</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> a</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> simple</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> one</span><span style='background-color:rgb(236.7957240343094,18.204275965690613,18.204275965690613)'> ,</span><span style='background-color:rgb(177.5893758237362,77.41062417626381,77.41062417626381)'> allan</span><span style='background-color:rgb(177.5893758237362,77.41062417626381,77.41062417626381)'> thought</span><span style='background-color:rgb(177.5893758237362,77.41062417626381,77.41062417626381)'> it</span><span style='background-color:rgb(177.5893758237362,77.41062417626381,77.41062417626381)'> had</span><span style='background-color:rgb(177.5893758237362,77.41062417626381,77.41062417626381)'> potential</span><span style='background-color:rgb(177.5893758237362,77.41062417626381,77.41062417626381)'> .</span><span style='background-color:rgb(177.5893758237362,77.41062417626381,77.41062417626381)'> besides</span><span style='background-color:rgb(177.5893758237362,77.41062417626381,77.41062417626381)'> ,</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> the</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> public</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> liked</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> a</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> good</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> ,</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> simple</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> romance</span><span style='background-color:rgb(168.78087848424911,86.21912151575089,86.21912151575089)'> .</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> perhaps</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> this</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> will</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> be</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> more</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> sale</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> ##able</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> ,</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> he</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> thought</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> and</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> began</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> to</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> write</span><span style='background-color:rgb(110.54690569639206,144.45309430360794,144.45309430360794)'> .</span> they gazed at each other , lost in the rap ##ture of love based so deeply within their hearts that they had never seen it before .<span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> "</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> what</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> about</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> roger</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> ?</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> "</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> she</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> asked</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> ,</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> knowing</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> that</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> the</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> answer</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> no</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> longer</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> mattered</span><span style='background-color:rgb(111.181760430336,143.818239569664,143.818239569664)'> .</span> [SEP]<span style='background-color:rgb(0.0,255.0,255.0)'> name</span><span style='background-color:rgb(0.0,255.0,255.0)'> few</span><span style='background-color:rgb(0.0,255.0,255.0)'> objects</span><span style='background-color:rgb(0.0,255.0,255.0)'> said</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> in</span><span style='background-color:rgb(0.0,255.0,255.0)'> or</span><span style='background-color:rgb(0.0,255.0,255.0)'> on</span><span style='background-color:rgb(0.0,255.0,255.0)'> allan</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> desk</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##r</span><span style='background-color:rgb(0.0,255.0,255.0)'> [SEP]</span> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

tensor([0.0627, 0.0610], device='cuda:0', grad_fn=<SelectBackward0>)


 [CLS] allan sat down at his desk and pulled the chair in close . opening a side drawer ,<span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> he</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> took</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> out</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> a</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> piece</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> of</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> paper</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> and</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> his</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> ink</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> ##pot</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> .</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> after</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> filling</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> his</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> pen</span><span style='background-color:rgb(109.09226775169373,145.90773224830627,145.90773224830627)'> ,</span> allan looked at his paper in the orange glow from the lantern set back in the desk ' s right - hand corner . his pen cast a forbid ##ding line of shadow slant ##ing across the page , echoing the ink ##y darkness crouch ##ing in the edges of the lantern ' s struggling glow . the only other illumination came from a lu ##rid moonlight filtered through thin branches and clouds , casting its bone - pale glow onto the pine floor ##boards . allan unfolded another page ,<span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> this</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> one</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> crowded</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> with</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> ranks</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> of</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> letters</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> in</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> tight</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> formation</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> from</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> left</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> to</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> right</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> .</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> the</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> lines</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> of</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> letters</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> stepped</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> into</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> their</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> divisions</span><span style='background-color:rgb(204.37600046396255,50.623999536037445,50.623999536037445)'> ,</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> in</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> the</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> shape</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> of</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> a</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> story</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> '</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> s</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> outline</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> :</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> the</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> loose</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> ,</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> dry</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> skeleton</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> of</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> a</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> tale</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> lay</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> exposed</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> beneath</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> their</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> feet</span><span style='background-color:rgb(175.63563466072083,79.36436533927917,79.36436533927917)'> ,</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> awaiting</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> tend</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> ##ons</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> ,</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> muscle</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> and</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> blushing</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> skin</span><span style='background-color:rgb(86.45198822021484,168.54801177978516,168.54801177978516)'> .</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> allan</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> reviewed</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> the</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> troops</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> ,</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> all</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> prepared</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> to</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> di</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> ##se</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> ##mba</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> ##rk</span><span style='background-color:rgb(135.54998204112053,119.45001795887947,119.45001795887947)'> ,</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> their</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> task</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> to</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> form</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> the</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> tale</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> of</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> a</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> young</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> man</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> returning</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> home</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> from</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> life</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> abroad</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> to</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> find</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> his</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> childhood</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> friend</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> a</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> bride</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> to</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> -</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> be</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> ,</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> thus</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> upset</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> ##ting</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> the</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> apple</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> cart</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> of</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> his</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> life</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> '</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> s</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> plan</span><span style='background-color:rgb(10.650752484798431,244.34924751520157,244.34924751520157)'> ,</span> clarified – of course – by his very time away from her he loved best . although the concept was a simple one ,<span style='background-color:rgb(128.58189314603806,126.41810685396194,126.41810685396194)'> allan</span><span style='background-color:rgb(128.58189314603806,126.41810685396194,126.41810685396194)'> thought</span><span style='background-color:rgb(128.58189314603806,126.41810685396194,126.41810685396194)'> it</span><span style='background-color:rgb(128.58189314603806,126.41810685396194,126.41810685396194)'> had</span><span style='background-color:rgb(128.58189314603806,126.41810685396194,126.41810685396194)'> potential</span><span style='background-color:rgb(128.58189314603806,126.41810685396194,126.41810685396194)'> .</span><span style='background-color:rgb(128.58189314603806,126.41810685396194,126.41810685396194)'> besides</span><span style='background-color:rgb(128.58189314603806,126.41810685396194,126.41810685396194)'> ,</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> the</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> public</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> liked</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> a</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> good</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> ,</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> simple</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> romance</span><span style='background-color:rgb(87.1157518029213,167.8842481970787,167.8842481970787)'> .</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> perhaps</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> this</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> will</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> be</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> more</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> sale</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> ##able</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> ,</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> he</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> thought</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> and</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> began</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> to</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> write</span><span style='background-color:rgb(1.519918441772461e-05,254.99998480081558,254.99998480081558)'> .</span> they gazed at each other , lost in the rap ##ture of love based so deeply within their hearts that they had never seen it before .<span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> "</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> what</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> about</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> roger</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> ?</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> "</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> she</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> asked</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> ,</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> knowing</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> that</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> the</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> answer</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> no</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> longer</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> mattered</span><span style='background-color:rgb(37.8537055850029,217.1462944149971,217.1462944149971)'> .</span> [SEP]<span style='background-color:rgb(0.0,255.0,255.0)'> name</span><span style='background-color:rgb(0.0,255.0,255.0)'> few</span><span style='background-color:rgb(0.0,255.0,255.0)'> objects</span><span style='background-color:rgb(0.0,255.0,255.0)'> said</span><span style='background-color:rgb(0.0,255.0,255.0)'> to</span><span style='background-color:rgb(0.0,255.0,255.0)'> be</span><span style='background-color:rgb(0.0,255.0,255.0)'> in</span><span style='background-color:rgb(0.0,255.0,255.0)'> or</span><span style='background-color:rgb(0.0,255.0,255.0)'> on</span><span style='background-color:rgb(0.0,255.0,255.0)'> allan</span><span style='background-color:rgb(0.0,255.0,255.0)'> '</span><span style='background-color:rgb(0.0,255.0,255.0)'> s</span><span style='background-color:rgb(0.0,255.0,255.0)'> desk</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##r</span><span style='background-color:rgb(0.0,255.0,255.0)'> [SEP]</span> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [23]:
expln.logits.shape

torch.Size([1, 2])

In [24]:
preds = expln.logits.argmax(-1)
preds, labels

(tensor([0], device='cuda:0'), tensor([0], device='cuda:0'))

# IDG

## PLS

In [26]:
import argparse
# import time
# import torch
# import numpy as np
import math
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
sys.path.append('../../lib/ExplanationSearch')
from explanation_methods_and_eval.src.models.parallel_local_search import PLSExplainer

In [27]:
class Args(argparse.Namespace):
    gpu = 0
    seed = 0
    explanation_sparsity = 0.2
    num_search = 1000
    batch_size = 10
    num_restarts = 10
    objective = "suff"

args=Args()

In [28]:
# parse args and set cuda device
device = torch.device(f"cuda:{args.gpu}")
args.device = device
torch.cuda.set_device(device)
np.random.seed(args.seed)
torch.random.manual_seed(args.seed)

# in PLS, there's a np.exp(c / temp) where temp is 0, so ignore this warning. (temp can be changed to change explore/exploit dynamics)
np.seterr(all="ignore") 

# define weight of evidence function for use in objective_function    
def np_woe(p):
    return np.log(p / (1-p))

# define objective function here. will use this to define an f(x) objective function to give to PLSExplainer class
def objective_function(args, explanation, always_keep_idx, task_model, orig_pred_prob, orig_input_kwargs):
    '''
    computes suff or comp objective on the current explanation                
    - explanation : np.ndarray of shape (r,d) where r is number of parallel runs and d is number of tokens (excluding always_keep tokens described next). 
    - always_keep_idx : binary array of len d', where d' is the number of tokens including tokens to be always kept, like special tokens
    - task_model: classifier that returns tuple with logits as SECOND element (after e.g. the loss)
    - orig_pred_prob: the predicted probability (for the predicted class) of the classifier on the current data point
    - orig_input_kwargs: dict with kwargs used to obtain orig_pred_prob, including 'attention_mask' item
    returns the suff/comp of task_model computed on document using attention_mask=mask, as well as the weight of evidence version of the objective
    ''' 
    # split input into batches if too many parallel runs (i.e. parallel states = parallel data points) to fit into memory
    num_batches = max(1, math.ceil(args.num_restarts / args.batch_size)) # num_restarts is num parallel runs
    num_tokens = orig_input_kwargs['input_ids'].size(-1)
    explanations = np.array_split(explanation, indices_or_sections=num_batches)
    obj_vals, obj_val_woes = [], []
    # repeat original data point batch_size times
    stacked_kwargs = {
        k : v.expand([args.batch_size] + list(v.shape)[-1:]).squeeze(-1).clone() for k,v in orig_input_kwargs.items() # clone to reallocate memory / squeeze() for labels special case
    }
    # get eligible for removal idx
    eligible_for_removal = torch.ones(num_tokens).to(args.device)
    eligible_for_removal[always_keep_idx] = 0.
    eligible_for_removal_idx = torch.where(eligible_for_removal)[0]
    for explanation in explanations:
        new_attention_mask = torch.tensor(explanation).long().to(args.device)
        stacked_kwargs['attention_mask'][:,eligible_for_removal_idx] = new_attention_mask
        outputs = task_model(**stacked_kwargs)
        pred_probs = torch.softmax(outputs[1], dim=-1)
        pred_prob = torch.gather(pred_probs, 1, stacked_kwargs['labels'].reshape(-1,1)).detach().cpu().numpy()
        if args.objective == 'suff':
            obj_val = orig_pred_prob - pred_prob
            obj_val_woe = np_woe(orig_pred_prob) - np_woe(pred_prob)
        if args.objective == 'comp':
            obj_val = -(orig_pred_prob - pred_prob)
            obj_val_woe = -(np_woe(orig_pred_prob) - np_woe(pred_prob))
        obj_vals.append(obj_val)
        obj_val_woes.append(obj_val_woe)
    obj_val = np.concatenate(obj_vals).reshape(-1)
    obj_val_woe = np.concatenate(obj_val_woes).reshape(-1)
    return obj_val, obj_val_woe

In [33]:
backbone_model = backbone_model.to(args.device).eval()
special_tokens = [processor.bos_token_id, processor.sep_token_id, processor.eos_token_id, processor.pad_token_id]    

# tokenized_input_seq_pair = processor.encode_plus(premise, hypothesis, max_length=256, return_token_type_ids=True, truncation=True)
model_kwargs = {
    'input_ids' : inputs,
    'token_type_ids' : token_type_ids,
    'attention_mask': attention_mask,
    'labels' : labels,
}
model_kwargs

{'input_ids': tensor([[  101,  8926,  2938,  2091,  2012,  2010,  4624,  1998,  2766,  1996,
           3242,  1999,  2485,  1012,  3098,  1037,  2217, 13065,  1010,  2002,
           2165,  2041,  1037,  3538,  1997,  3259,  1998,  2010, 10710, 11008,
           1012,  2044,  8110,  2010,  7279,  1010,  8926,  2246,  2012,  2010,
           3259,  1999,  1996,  4589,  8652,  2013,  1996, 12856,  2275,  2067,
           1999,  1996,  4624,  1005,  1055,  2157,  1011,  2192,  3420,  1012,
           2010,  7279,  3459,  1037, 27206,  4667,  2240,  1997,  5192, 27474,
           2075,  2408,  1996,  3931,  1010, 17142,  1996, 10710,  2100,  4768,
          21676,  2075,  1999,  1996,  7926,  1997,  1996, 12856,  1005,  1055,
           8084,  8652,  1012,  1996,  2069,  2060, 21203,  2234,  2013,  1037,
          11320, 14615, 11986, 21839,  2083,  4857,  5628,  1998,  8044,  1010,
           9179,  2049,  5923,  1011,  5122,  8652,  3031,  1996,  7222,  2723,
          15271,  1012,  89

In [32]:
inputs.shape, attention_mask.shape, token_type_ids.shape, labels.shape

(torch.Size([1, 512]),
 torch.Size([1, 512]),
 torch.Size([1, 512]),
 torch.Size([1]))

In [35]:
always_keep_idx = np.argwhere([_id in special_tokens for _id in model_kwargs['input_ids'].squeeze()]).reshape(-1)
always_keep_idx = torch.Tensor(always_keep_idx).long().to(args.device)
num_spcl_tokens = len(always_keep_idx)
search_dimensionality = model_kwargs['input_ids'].shape[-1] - num_spcl_tokens

outputs = backbone_model(**model_kwargs)
outputs

SequenceClassifierOutput(loss=tensor(0.0008, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 3.9219, -3.1765]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [36]:
pred_probs = torch.softmax(outputs[1], dim=1)[0]
pred_class = torch.argmax(pred_probs)
pred_prob = pred_probs[pred_class].item()

In [37]:
pred_prob

0.999174177646637

In [41]:
def current_objective_function(explanation):
    return objective_function(
        args=args, 
        explanation=explanation, 
        always_keep_idx=always_keep_idx,
        task_model=backbone_model, 
        orig_pred_prob=pred_prob, 
        orig_input_kwargs=model_kwargs
    )

In [42]:
# initialize PLS 
PLS = PLSExplainer(
    objective_function=current_objective_function, 
    target_sparsity=args.explanation_sparsity, 
    eval_budget=args.num_search,
    dimensionality=search_dimensionality,
    restarts=args.num_restarts, # num parallel runs
    temp_decay=0, # temp in simulated annealing (0 <= x <= 1). set to higher value for more exploration early in search
    search_space='exact_k',
    no_duplicates=True # avoid duplicate evaluations of objective function
)

In [43]:
explanations, obj_values, obj_woe_values = PLS.run()

In [44]:
explanations.shape, explanations

((1, 353),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
         0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
         1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
         0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.,

In [46]:
print_str = ""
for i in range(explanations.shape[1]):
    expln_val = explanations[0][i].item()
    if expln_val == 0:
        print_str += f" {tokens[i]}"
    else:
        print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
printmd(print_str)

<span style='background-color:rgb(0.0,255.0,255.0)'> [CLS]</span> allan sat down at his desk and pulled the<span style='background-color:rgb(0.0,255.0,255.0)'> chair</span> in close . opening a side drawer<span style='background-color:rgb(0.0,255.0,255.0)'> ,</span> he took out a<span style='background-color:rgb(0.0,255.0,255.0)'> piece</span> of paper and his ink ##pot . after filling his<span style='background-color:rgb(0.0,255.0,255.0)'> pen</span> , allan looked at his paper in the orange glow<span style='background-color:rgb(0.0,255.0,255.0)'> from</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span><span style='background-color:rgb(0.0,255.0,255.0)'> lantern</span> set back<span style='background-color:rgb(0.0,255.0,255.0)'> in</span> the<span style='background-color:rgb(0.0,255.0,255.0)'> desk</span> '<span style='background-color:rgb(0.0,255.0,255.0)'> s</span> right - hand corner . his<span style='background-color:rgb(0.0,255.0,255.0)'> pen</span><span style='background-color:rgb(0.0,255.0,255.0)'> cast</span> a forbid ##ding line of shadow slant ##ing across the page , echoing<span style='background-color:rgb(0.0,255.0,255.0)'> the</span> ink<span style='background-color:rgb(0.0,255.0,255.0)'> ##y</span> darkness crouch ##ing in the edges<span style='background-color:rgb(0.0,255.0,255.0)'> of</span> the lantern ' s<span style='background-color:rgb(0.0,255.0,255.0)'> struggling</span> glow . the<span style='background-color:rgb(0.0,255.0,255.0)'> only</span> other<span style='background-color:rgb(0.0,255.0,255.0)'> illumination</span> came from a lu ##rid<span style='background-color:rgb(0.0,255.0,255.0)'> moonlight</span> filtered through thin branches and clouds , casting its bone -<span style='background-color:rgb(0.0,255.0,255.0)'> pale</span> glow onto the pine floor ##boards .<span style='background-color:rgb(0.0,255.0,255.0)'> allan</span> unfolded<span style='background-color:rgb(0.0,255.0,255.0)'> another</span> page ,<span style='background-color:rgb(0.0,255.0,255.0)'> this</span> one<span style='background-color:rgb(0.0,255.0,255.0)'> crowded</span> with<span style='background-color:rgb(0.0,255.0,255.0)'> ranks</span> of letters in tight formation from left to right . the<span style='background-color:rgb(0.0,255.0,255.0)'> lines</span> of letters stepped into their divisions , in the shape<span style='background-color:rgb(0.0,255.0,255.0)'> of</span> a story ' s<span style='background-color:rgb(0.0,255.0,255.0)'> outline</span> : the loose<span style='background-color:rgb(0.0,255.0,255.0)'> ,</span> dry skeleton of a tale lay exposed beneath their feet<span style='background-color:rgb(0.0,255.0,255.0)'> ,</span> awaiting tend ##ons , muscle and blushing skin . allan reviewed the troops , all prepared to di ##se<span style='background-color:rgb(0.0,255.0,255.0)'> ##mba</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##rk</span> , their<span style='background-color:rgb(0.0,255.0,255.0)'> task</span> to form<span style='background-color:rgb(0.0,255.0,255.0)'> the</span> tale of<span style='background-color:rgb(0.0,255.0,255.0)'> a</span> young man returning home from life abroad to find<span style='background-color:rgb(0.0,255.0,255.0)'> his</span> childhood friend a bride to - be , thus upset ##ting the apple cart of<span style='background-color:rgb(0.0,255.0,255.0)'> his</span> life ' s plan , clarified<span style='background-color:rgb(0.0,255.0,255.0)'> –</span> of<span style='background-color:rgb(0.0,255.0,255.0)'> course</span><span style='background-color:rgb(0.0,255.0,255.0)'> –</span> by<span style='background-color:rgb(0.0,255.0,255.0)'> his</span> very time away from her<span style='background-color:rgb(0.0,255.0,255.0)'> he</span> loved<span style='background-color:rgb(0.0,255.0,255.0)'> best</span> .<span style='background-color:rgb(0.0,255.0,255.0)'> although</span> the concept was a simple one , allan<span style='background-color:rgb(0.0,255.0,255.0)'> thought</span> it<span style='background-color:rgb(0.0,255.0,255.0)'> had</span> potential . besides ,<span style='background-color:rgb(0.0,255.0,255.0)'> the</span> public<span style='background-color:rgb(0.0,255.0,255.0)'> liked</span> a good , simple romance<span style='background-color:rgb(0.0,255.0,255.0)'> .</span> perhaps<span style='background-color:rgb(0.0,255.0,255.0)'> this</span> will be more sale ##able<span style='background-color:rgb(0.0,255.0,255.0)'> ,</span> he thought and began<span style='background-color:rgb(0.0,255.0,255.0)'> to</span> write . they gazed at each other<span style='background-color:rgb(0.0,255.0,255.0)'> ,</span> lost<span style='background-color:rgb(0.0,255.0,255.0)'> in</span> the<span style='background-color:rgb(0.0,255.0,255.0)'> rap</span> ##ture of love<span style='background-color:rgb(0.0,255.0,255.0)'> based</span><span style='background-color:rgb(0.0,255.0,255.0)'> so</span> deeply within their<span style='background-color:rgb(0.0,255.0,255.0)'> hearts</span> that they<span style='background-color:rgb(0.0,255.0,255.0)'> had</span> never<span style='background-color:rgb(0.0,255.0,255.0)'> seen</span> it before<span style='background-color:rgb(0.0,255.0,255.0)'> .</span> " what about roger ?<span style='background-color:rgb(0.0,255.0,255.0)'> "</span> she asked , knowing<span style='background-color:rgb(0.0,255.0,255.0)'> that</span><span style='background-color:rgb(0.0,255.0,255.0)'> the</span> answer no longer mattered . [SEP] name few<span style='background-color:rgb(0.0,255.0,255.0)'> objects</span><span style='background-color:rgb(0.0,255.0,255.0)'> said</span> to<span style='background-color:rgb(0.0,255.0,255.0)'> be</span> in or on allan ' s<span style='background-color:rgb(0.0,255.0,255.0)'> desk</span> |<span style='background-color:rgb(0.0,255.0,255.0)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span>

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from scipy.ndimage.filters import gaussian_filter
from exlib.evaluators.common import Evaluator, convert_idx_masks_to_bool


class CompSuff(Evaluator):

    def __init__(self, model, 
                 mode: str='comp', 
                 k_fraction: list=[0.01, 0.05, 0.10, 0.20, 0.50], 
                 postprocess=None,
                 task_type='cls'):
        """Create deletion/insertion metric instance.
        Args:
            model (nn.Module): Black-box model being explained.
            mode (str): 'comp' or 'suff'.
            k_fraction (float): number of pixels modified per one iteration.
            substrate_fn (func): a mapping from old pixels to new pixels.
        """
        super(CompSuff, self).__init__(model, postprocess)
        
        assert mode in ['comp', 'suff']
        assert task_type in ['cls', 'reg']
        self.mode = mode
        self.task_type = task_type
        self.k_fraction = k_fraction

    def forward(self, X, Z, kwargs=None, verbose=0, save_to=None):
        """Run metric on one image-saliency pair.
            Args:
                X = img_tensor (Tensor): normalized image tensor. (bsz, n_channel, img_dim1, img_dim2)
                Z = explanation (Tensor): saliency map. (bsz, 1, img_dim1, img_dim2)
                verbose (int): in [0, 1, 2].
                    0 - return list of scores.
                    1 - also plot final step.
                    2 - also plot every step and print 2 top classes.
                save_to (str): directory to save every step plots to.
            Return:
                scores (Tensor): Array containing scores at every step.
        """
        self.model.eval()

        scores = []

        for i in range(X.shape[0]):
            img_tensor = X[i:i+1]
            explanation = Z[i:i+1]
            if kwargs is not None:
                kwargs_curr = {k: v[i:i+1] for k, v in kwargs.items()}
            else:
                kwargs_curr = None
            if len(X.shape) == 4: # image
                bsz, n_channel, img_dim1, img_dim2 = X.shape
                HW = img_dim1 * img_dim2
                model_type = 'image'
            else: # text
                bsz, seq_len = X.shape
                HW = seq_len
                n_channel = 1
                model_type = 'text'
            with torch.no_grad():
                
                if kwargs_curr:
                    pred = self.model(img_tensor, **kwargs_curr)
                else:
                    pred = self.model(img_tensor)
                if self.postprocess is not None:
                    pred = self.postprocess(pred)
            pred = torch.softmax(pred, dim=-1)
            top, c = torch.max(pred, 1)

            for k_fraction in self.k_fraction:
                if model_type == 'image':
                    step = int(k_fraction * HW)
                    non_pad_HW = HW
                else:
                    non_pad_HW = (X[i] != 0).sum(-1)
                    step = int(k_fraction * non_pad_HW)
                    if step == 0:
                        step = 1

                if self.mode == 'comp':
                    start = img_tensor.clone()
                    finish = torch.zeros_like(img_tensor)
                elif self.mode == 'suff':
                    start = torch.zeros_like(img_tensor)
                    finish = img_tensor.clone()

                # start[start < 0] = 0.0
                # start[start > 1] = 1.0
                # finish[finish < 0] = 0.0
                # finish[finish > 1] = 1.0

                # Coordinates of pixels in order of decreasing saliency
                # import pdb
                # pdb.set_trace()
                explanation = explanation.reshape(1, 1, -1)
                t_r = explanation[:,:,:non_pad_HW].reshape(1, -1, non_pad_HW)
                salient_order = torch.argsort(t_r, dim=-1)
                salient_order = torch.flip(salient_order, [1, 2])
                # if self.mode == 'suff':
                #     coords_top_k = salient_order[:, :, :step]
                #     coords = coords_top_k
                # else:
                # always choose bottom to flip?
                coords_bottom_k = salient_order[:, :, non_pad_HW - 1 - step:]
                coords = coords_bottom_k
                batch_indices = torch.arange(1).view(-1, 1, 1).to(coords.device)
                channel_indices = torch.arange(n_channel).view(1, -1, 1).to(coords.device)

                start.reshape(1, n_channel, HW)[batch_indices, 
                                                channel_indices, 
                                                coords] = finish.reshape(1, n_channel, HW)[batch_indices,
                                                                                            channel_indices, 
                                                                                            coords]
                with torch.no_grad():
                    if kwargs_curr:
                        # import pdb
                        # pdb.set_trace()
                        # inputs_new = img_tensor[start == 1].unsqueeze(0)
                        # kwargs_curr_new = {k: v[start == 1].unsqueeze(0) for k, v in kwargs_curr.items()}
                        # inputs_new_pad = F.pad(inputs_new, (0, 512 - len(inputs_new)))
                        # kwargs_curr_new_pad = {k: F.pad(v, (0, 512 - len(v))) for k, v in kwargs_curr_new.items()}
                        pred_mod = self.model(start, **kwargs_curr)
                    else:
                        # inputs_new = img_tensor[start == 1]
                        # inputs_new_pad = F.pad(inputs_new, (0, 512 - len(inputs_new)))
                        pred_mod = self.model(start)
                    if self.postprocess is not None:
                        pred_mod = self.postprocess(pred_mod)
                pred_mod = torch.softmax(pred_mod, dim=-1)
                score = pred[0, c] - pred_mod[0, c]
                scores.append(score)
        scores = torch.cat(scores)
        scores = scores.reshape(bsz, -1).sum(-1) / len(self.k_fraction)
        return scores

/tmp/ipykernel_4114/2943455553.py:7: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [55]:
comp_evaluator = CompSuff(model, mode='comp')
with torch.no_grad():
    comp_score = comp_evaluator(inputs, expln.attributions, kwargs=)
inputs.shape, labels.shape, comp_score

IndexError: too many indices for tensor of dimension 1